# Information Theory

## Shamir's Secret Sharing

###### Odile, Gaspard, Antoine

## Outline

1. [Theory of SSS](#theory)
2. [Examples in low dimension.](#example)
3. [The code: a simulation of secret sharing.](#code)

## <a id="theory"></a>Theory of SSS

## <a id="example"></a>Examples in low dimension.

## <a id="code"></a>The code: a simulation of secret sharing.

The base unit of an evolutionary algorithm is the individual. An individual represents a single solution to the problem we want to solve. We'll start simple with binary individuals, where genes are represented by bits. DNA in biological organisms is mostly base 4, being represented by 4 different amino acids. We'll use a base 2 representation of 1s and 0s, but the interpretation of these binary strings could lead to any complex organism. Using binary strings allows us to discuss the theoretical analysis of evolutionary algorithms, which have mostly been studied on binary genomes.